In [1]:
from importlib import import_module
import pandas as pd 
from datetime import datetime, timedelta
from os import listdir
import plotly.express as px
from os.path import isfile, join
import numpy as np
import sys
#sys.path.insert(1, 'C:/Users/maperezr/proyectos/fs-emails')
#from adm_email import ADM_MAIL
#from input_planillas import f_miguel as fm
dt_string = datetime.today().strftime('%d%m%y')
pd.set_option('display.max_columns', 500)

In [2]:
fecha_inicio = "2022-08-01"
fecha_final = "2022-11-29"
f_inicio = datetime.strptime(fecha_inicio, '%Y-%m-%d')
f_final = datetime.strptime(fecha_final, '%Y-%m-%d')

In [3]:
lista_fechas = [(f_inicio + timedelta(days=d)).strftime('%y%m%d') for d in range((f_final - f_inicio).days + 1)] 
len(lista_fechas)

121

In [4]:
with open('config\path.txt', "r") as tf:
        path = tf.read()
        path = path.replace("\\","/")
tf.close()
alertas = pd.DataFrame()
for i in listdir(path):
        files_store= pd.DataFrame()
        if i.startswith('JPPs - Análisis de notas crédito - '):
                new_path= f"{path}/{i}"
                files_names = [f for f in listdir(new_path) if isfile(join(new_path, f))]
                for j in (files_names):
                        if j.endswith('.xlsx'):
                                for i in lista_fechas:
                                        if j.startswith(i):
                                                if '_cm' in j:
                                                        pass
                                                else:
                                                        file = pd.read_excel(f'{new_path}/{j}',sheet_name="Alertas",dtype=str)
                                                        file["fecha"] = i
                                                        files_store = pd.concat([files_store,file])
        alertas = pd.concat([alertas, files_store])  


In [8]:
alertas.columns

Index([             'Tipo alerta',                'Cautoriza',
                     'Desc_local',              'Dcompra_nvo',
                  'Nterminal_nvo',           'Nsecuencia_nvo',
                           'Hora',             'Cod_Empleado',
                  'Num_Documento',          'nombre_completo',
                          'Cargo',                      'SKU',
                      'Qcantidad',        'Costo_NC-Empleado',
                     'Grabación?',                 'Cliente?',
                      'Producto?',            'Observaciones',
                          'fecha',                        ' ',
                    'Unnamed: 18',              'Unnamed: 19',
                    'Unnamed: 20',              'Unnamed: 21',
                     'gravacion?',              'Unnamed: 17',
                    'OBSERVACION',              'Observacion',
                   'Observacione',                        's',
                             'si',               'Unnam

In [14]:
#alertas.to_excel('output/inf_alerta_mensualult.xlsx')

In [ ]:
#ruta=r"C:\Users\valetorresr\Proyectos\nc_santiago\output\inf_alerta_mensual1.xlsx"
#alertas=pd.read_excel(ruta)

In [9]:
alertas.loc[(alertas["Grabación?"].notna()) & (alertas["Cliente?"].notna()) & (alertas["Producto?"].notna()), "Indicador"] = "Diligenciado"
alertas.loc[alertas["Indicador"] != "Diligenciado", "Indicador"] = "No diligenciado"

In [10]:
alertas["Grabación?"]= alertas["Grabación?"].str.upper()
alertas["Cliente?"]= alertas["Cliente?"].str.upper()
alertas["Producto?"]= alertas["Producto?"].str.upper()

In [11]:
alertas.loc[(alertas["Grabación?"]=="SI") | (alertas["Grabación?"]=="SI OK") | (alertas["Grabación?"]=="SIS")| (alertas["Grabación?"]=="OK")| (alertas["Grabación?"]=="SI OK ")| (alertas["Grabación?"]=="SI ") , "Grab"] = "SI"
alertas.loc[(alertas["Grabación?"]=="NO") | (alertas["Grabación?"]=="ON") | (alertas["Grabación?"]=="N0")| (alertas["Grabación?"]=="NO "), "Grab"] = "NO"
alertas.loc[(alertas["Grab"]!="NO") & (alertas["Grab"]!="SI"), "Grab"] = "VACIO"

alertas.loc[(alertas["Cliente?"]=="SI") | (alertas["Cliente?"]=="SI OK") | (alertas["Cliente?"]=="SIS")| (alertas["Cliente?"]=="OK")| (alertas["Cliente?"]=="SI OK ")| (alertas["Cliente?"]=="SI ") , "Clien"] = "SI"
alertas.loc[(alertas["Cliente?"]=="NO") | (alertas["Cliente?"]=="ON") | (alertas["Cliente?"]=="N0")| (alertas["Cliente?"]=="NO "), "Clien"] = "NO"
alertas.loc[(alertas["Clien"]!="NO") & (alertas["Clien"]!="SI"), "Clien"] = "VACIO"

alertas.loc[(alertas["Producto?"]=="SI") | (alertas["Producto?"]=="SI OK") | (alertas["Producto?"]=="SIS")| (alertas["Producto?"]=="OK")| (alertas["Producto?"]=="SI OK ")| (alertas["Producto?"]=="SI ") , "Prod"] = "SI"
alertas.loc[(alertas["Producto?"]=="NO") | (alertas["Producto?"]=="ON") | (alertas["Producto?"]=="N0")| (alertas["Producto?"]=="NO "), "Prod"] = "NO"
alertas.loc[(alertas["Prod"]!="NO") & (alertas["Prod"]!="SI"), "Prod"] = "VACIO"

In [12]:
alertas.loc[(alertas["Grab"]=="SI") & (alertas["Clien"]=="NO") & (alertas["Prod"]=="NO"), "Falta_CL_PR"] = "Alerta"
alertas.loc[(alertas["Grab"]=="SI") & (alertas["Clien"]=="NO") & (alertas["Prod"]=="SI"), "Falta_CL"] = "Alerta"
alertas.loc[(alertas["Grab"]=="SI") & (alertas["Clien"]=="SI") & (alertas["Prod"]=="NO"), "Falta_PR"] = "Alerta"

In [13]:
alertas['Falta_CL_PR']=alertas['Falta_CL_PR'].fillna('No Alerta')
alertas['Falta_CL']=alertas['Falta_CL'].fillna('No Alerta')
alertas['Falta_PR']=alertas['Falta_PR'].fillna('No Alerta')

In [11]:
alertas[['Tipo alerta','Cautoriza', 'Desc_local', 'Dcompra_nvo',
'Nterminal_nvo', 'Nsecuencia_nvo', 'Hora', 'Cod_Empleado',
'Num_Documento', 'nombre_completo', 'Cargo', 'SKU', 'Qcantidad',
'Costo_NC-Empleado', 'Grabación?', 'Cliente?', 'Producto?','Indicador','Grab','Clien','Prod','Falta_CL_PR','Falta_CL','Falta_PR']].to_csv('output/inf_alerta_mensual8111.csv')

In [148]:
alertas_x_hora = alertas.loc[alertas['Tipo alerta'] == 'Alerta x Hora']
alertas_x_monto = alertas.loc[alertas['Tipo alerta'] == 'Alerta x monto']

In [149]:
alertas_x_hora.head()

,Tipo alerta,Cautoriza,Desc_local,Dcompra_nvo,Nterminal_nvo,Nsecuencia_nvo,Hora,Cod_Empleado,Num_Documento,nombre_completo,Cargo,SKU,Qcantidad,Costo_NC-Empleado,Grabación?,Cliente?,Producto?,Observaciones,fecha,,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,gravacion?,Unnamed: 17,OBSERVACION,Observacion,Observacione,s,si,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Hay Grabación?,Esta el cliente?,Se revisa el Producto?,Se dejan los Producto?,OBSERVACIONES,NaN,Unnamed: 22,Unnamed: 23,,OMT,56676,Indicador,Grab,Clien,Prod,Falta_CL_PR,Falta_CL,Falta_PR
10,Alerta x Hora,1080911279,ACQUA,2022-09-02 00:00:00,33,6693,2112,102636,1110504794,MOTA MUÑOZ ANGIE YULIETH,ESTUDIANTE EN PRACTICA,2,-2,1174890,SI,SI,SI,NaN,220902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
19,Alerta x Hora,1082520622,ACQUA,2022-11-15 00:00:00,31,5626,2101,134694,1121925805,LADINO TEJEIRO ANA MARIA,AUXILIAR DE PREVENCION Y PERDIDAS,1,-1,71994,SI,SI,SI,NaN,221115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
20,Alerta x Hora,1082521155,ACQUA,2022-11-15 00:00:00,29,1239,2124,170305,1110571354,ZABALA PAEZ SHIRLEY CAROLINA,VENDEDOR INTEGRAL,2,-2,1549890,SI,SI,SI,NaN,221115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
21,Alerta x Hora,1082521568,ACQUA,2022-11-15 00:00:00,6,892,2149,118540,65719613,YATE LAVAO LUCY CATALINA,ASESOR DE COMPRAS,13,-13,463407,SI,SI,SI,NaN,221115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
22,Alerta x Hora,1082521810,ACQUA,2022-11-15 00:00:00,5,7608,2205,131555,1110566060,PEREA PINTO FRANK DAVID,VENDEDOR INTEGRAL,2,-2,1999901,SI,SI,SI,NaN,221115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta


In [150]:
alertas_x_hora.Cautoriza.nunique()

835

In [151]:
alertas_x_hora.columns

Index([             'Tipo alerta',                'Cautoriza',
                     'Desc_local',              'Dcompra_nvo',
                  'Nterminal_nvo',           'Nsecuencia_nvo',
                           'Hora',             'Cod_Empleado',
                  'Num_Documento',          'nombre_completo',
                          'Cargo',                      'SKU',
                      'Qcantidad',        'Costo_NC-Empleado',
                     'Grabación?',                 'Cliente?',
                      'Producto?',            'Observaciones',
                          'fecha',                        ' ',
                    'Unnamed: 18',              'Unnamed: 19',
                    'Unnamed: 20',              'Unnamed: 21',
                     'gravacion?',              'Unnamed: 17',
                    'OBSERVACION',              'Observacion',
                   'Observacione',                        's',
                             'si',               'Unnam

In [152]:

alertas_x_hora["fecha"] =  pd.to_datetime(alertas_x_hora.fecha, yearfirst=True)
al_x_hora = alertas_x_hora.loc[(alertas_x_hora["fecha"] >= fecha_inicio) & (alertas_x_hora["fecha"] <= fecha_final)].reset_index(drop=True)
al_x_hora["Grabación?"] = al_x_hora["Grabación?"].str.upper()
al_x_hora["Grabación?"] = al_x_hora["Grabación?"].fillna("nan")
al_x_hora.loc[(al_x_hora["Grabación?"].str.contains("NO")) | (al_x_hora["Grabación?"].notna()) & (al_x_hora["Cliente?"].notna()) & (al_x_hora["Producto?"].notna()),"Indicador"] = "Diligenciado"
al_x_hora.loc[al_x_hora["Grabación?"] =="nan", "Grabación?" ] = np.nan
print(f'cantidad de alertas generadas en el corte: {al_x_hora.shape[0]}')
al_x_hora.loc[(al_x_hora["Grabación?"].notna()) & (al_x_hora["Cliente?"].notna()) & (al_x_hora["Producto?"].notna()), "Indicador"] = "Diligenciado"
al_x_hora.loc[al_x_hora["Indicador"] != "Diligenciado", "Indicador"] = "No diligenciado"
fig_al_x_hora = al_x_hora.groupby(["Desc_local","Indicador"]).agg({"Cautoriza":"nunique",}).reset_index().sort_values("Cautoriza", ascending=False)
h_fig = px.bar(fig_al_x_hora, x="Desc_local", y="Cautoriza",color="Indicador",text="Cautoriza",color_discrete_map = {'No diligenciado':'rgb(170, 57, 57)','Diligenciado':'rgb(45, 136, 45)'}, title=f"Cantidad de alertas por hora revisadas por tienda <br>Corte: {fecha_inicio} hasta el {fecha_final}", labels={"Desc_local":"Local","Cautoriza":"Cantidad NCs"}) 
h_fig.update_layout(xaxis_categoryorder = 'total descending',legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.7))
h_fig.write_image(f"output/{dt_string}alerta_horas.jpg", height = 600, width = 1000)
h_fig.show()
al_x_hora.to_excel(f"output/{dt_string}_alertas_x_hora.xlsx", index=False)

C:\Users\valetorresr\AppData\Local\Temp\ipykernel_17876\3762780082.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



cantidad de alertas generadas en el corte: 835


In [153]:
al_x_hora.head()

,Tipo alerta,Cautoriza,Desc_local,Dcompra_nvo,Nterminal_nvo,Nsecuencia_nvo,Hora,Cod_Empleado,Num_Documento,nombre_completo,Cargo,SKU,Qcantidad,Costo_NC-Empleado,Grabación?,Cliente?,Producto?,Observaciones,fecha,,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,gravacion?,Unnamed: 17,OBSERVACION,Observacion,Observacione,s,si,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Hay Grabación?,Esta el cliente?,Se revisa el Producto?,Se dejan los Producto?,OBSERVACIONES,NaN,Unnamed: 22,Unnamed: 23,,OMT,56676,Indicador,Grab,Clien,Prod,Falta_CL_PR,Falta_CL,Falta_PR
0,Alerta x Hora,1080911279,ACQUA,2022-09-02 00:00:00,33,6693,2112,102636,1110504794,MOTA MUÑOZ ANGIE YULIETH,ESTUDIANTE EN PRACTICA,2,-2,1174890,SI,SI,SI,NaN,2022-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
1,Alerta x Hora,1082520622,ACQUA,2022-11-15 00:00:00,31,5626,2101,134694,1121925805,LADINO TEJEIRO ANA MARIA,AUXILIAR DE PREVENCION Y PERDIDAS,1,-1,71994,SI,SI,SI,NaN,2022-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
2,Alerta x Hora,1082521155,ACQUA,2022-11-15 00:00:00,29,1239,2124,170305,1110571354,ZABALA PAEZ SHIRLEY CAROLINA,VENDEDOR INTEGRAL,2,-2,1549890,SI,SI,SI,NaN,2022-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
3,Alerta x Hora,1082521568,ACQUA,2022-11-15 00:00:00,6,892,2149,118540,65719613,YATE LAVAO LUCY CATALINA,ASESOR DE COMPRAS,13,-13,463407,SI,SI,SI,NaN,2022-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta
4,Alerta x Hora,1082521810,ACQUA,2022-11-15 00:00:00,5,7608,2205,131555,1110566060,PEREA PINTO FRANK DAVID,VENDEDOR INTEGRAL,2,-2,1999901,SI,SI,SI,NaN,2022-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado,SI,SI,SI,No Alerta,No Alerta,No Alerta


In [49]:
with open('config\path.txt', "r") as tf:
        path = tf.read()
        path = path.replace("\\","/")
tf.close()
alertas_x_monto = pd.DataFrame()
for i in listdir(path):
        files_store= pd.DataFrame()
        if i.startswith('JPPs - Análisis de notas crédito - '):
                new_path= f"{path}/{i}"
                files_names = [f for f in listdir(new_path) if isfile(join(new_path, f))]
                for j in (files_names):
                        if j.endswith('.xlsx'):
                                for i in lista_fechas:
                                        if j.startswith(i):
                                                if '_cm' in j:
                                                        pass
                                                else:
                                                        file = pd.read_excel(f'{new_path}/{j}',sheet_name="Alertas",dtype=str)
                                                        file["fecha"] = i
                                                        files_store = pd.concat([files_store,file])
        alertas_x_monto = pd.concat([alertas_x_monto, files_store]) 

Alertas x monto despues de ultimo informe (20/feb/22)

In [154]:
alertas_x_monto["fecha"] =  pd.to_datetime(alertas_x_monto.fecha, yearfirst=True)
al_x_monto = alertas_x_monto.loc[(alertas_x_monto["fecha"] >= fecha_inicio) & (alertas_x_monto["fecha"] <= fecha_final)].reset_index(drop=True)
al_x_monto["Grabación?"] = al_x_monto["Grabación?"].str.upper()
al_x_monto["Grabación?"] = al_x_monto["Grabación?"].fillna("nan")
al_x_monto.loc[(al_x_monto["Grabación?"].str.contains("NO")) | (al_x_monto["Grabación?"].notna()) & (al_x_monto["Cliente?"].notna()) & (al_x_monto["Producto?"].notna()),"Indicador"] = "Diligenciado"
al_x_monto.loc[al_x_monto["Grabación?"] =="nan", "Grabación?" ] = np.nan
print(f'cantidad de alertas generadas en el corte: {al_x_monto.shape[0]}')
al_x_monto.loc[(al_x_monto["Grabación?"].notna()) & (al_x_monto["Cliente?"].notna()) & (al_x_monto["Producto?"].notna()), "Indicador"] = "Diligenciado"
al_x_monto.loc[al_x_monto["Indicador"] != "Diligenciado", "Indicador"] = "No diligenciado"
fig_al_x_monto = al_x_monto.groupby(["Desc_local","Indicador"]).agg({"Cautoriza":"nunique",}).reset_index().sort_values("Cautoriza", ascending=False)
h_fig = px.bar(fig_al_x_monto, x="Desc_local", y="Cautoriza",color="Indicador",text="Cautoriza",color_discrete_map = {'No diligenciado':'rgb(170, 57, 57)','Diligenciado':'rgb(45, 136, 45)'}, title=f"Cantidad de alertas por monto por tienda <br>Corte:{fecha_inicio} hasta el {fecha_final}", labels={"Desc_local":"Local","Cautoriza":"Cantidad NCs"}) 
h_fig.update_layout(xaxis_categoryorder = 'total descending',legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.7))
h_fig.write_image(f"output/{dt_string}alerta_monto.jpg", height = 600, width = 1000)
h_fig.show()
al_x_monto.to_excel(f"output/{dt_string}alertas_x_monto.xlsx", index=False)

C:\Users\valetorresr\AppData\Local\Temp\ipykernel_17876\2481296464.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



cantidad de alertas generadas en el corte: 46664


Calculos


In [51]:
hora_no_d = al_x_hora.loc[al_x_hora.Indicador == "No diligenciado", "Cautoriza" ].nunique()
hora_d = al_x_hora.loc[al_x_hora.Indicador == "Diligenciado", "Cautoriza" ].nunique()
monto_no_d = al_x_monto.loc[al_x_monto.Indicador == "No diligenciado", "Cautoriza" ].nunique()
monto_d = al_x_monto.loc[al_x_monto.Indicador == "Diligenciado", "Cautoriza" ].nunique()
total_alarmas = al_x_hora.shape[0] + al_x_monto.shape[0]
cantidad_diligenciados = hora_d + monto_d
print(f"Cantidad total de alarmas por hora: {al_x_hora.shape[0]}")
print(f"Cantidad total de alarmas por monto: {al_x_monto.shape[0]}")
print(f"Cantidad total de alarmas: {total_alarmas}")
print(f"Cantidad total no diligenciado: {hora_no_d + monto_no_d}")
print(f"Cantidad diligenciados: {cantidad_diligenciados}")
print(f"Porcentage de alertas diligenciadas: {(cantidad_diligenciados*100)/total_alarmas}")
alertas_hora = al_x_hora.shape[0]
alertas_montos = al_x_monto.shape[0]
alertas_totales = hora_no_d + monto_no_d


Cantidad total de alarmas por hora: 204
Cantidad total de alarmas por monto: 9159
Cantidad total de alarmas: 9363
Cantidad total no diligenciado: 3555
Cantidad diligenciados: 5676
Porcentage de alertas diligenciadas: 60.6215956424223


In [ ]:
email_content = f"""<p>Estimados:</p> 
<p>Por favor encuentren a continuación, el análisis de la gestión de diligenciamiento de los reportes de notas créditos según la tienda, para las alarmas por monto y hora, enviadas entre {fecha_inicio} hasta el {fecha_final}.</p>  
<p>En total se generaron {alertas_totales} alertas para las 26 tiendas, las cuales incluye {alertas_hora} alertas por hora y {alertas_montos} alertas por monto. Se evidencia que <strong><u>fueron diligenciadas por las tiendas 1445 alertas que corresponden al 48% del total de alertas</strong></u>.</p> 
<p>Así mismo, en las siguientes gráficas se presentan la cantidad de alertas a revisar por las tiendas y el estado del diligenciamiento correspondiente:</p>  
<img src="cid:alerta_horas"><br></td>
<img src="cid:alerta_montos"><br></td>
<p>El detalle de la información anterior se encuentra adjunto.</p>   
<p>Quedo atento en caso de que surja alguna duda.</p>  
<p>Saludos cordiales,</p> 
<br>
{fm.firma}
"""


: 

In [ ]:
img=  {'alerta_horas' : f'output/{dt_string}alerta_horas.jpg', 'alerta_montos':f'output/{dt_string}alerta_monto.jpg'}

: 

In [ ]:
print(f'{dt_string}_alertas_x_hora.xlsx')

: 

In [ ]:
mail = ADM_MAIL()
mail.attach_text(email_content)
mail.attach_image(img)
listado_archive = [f'output\{dt_string}_alertas_x_hora.xlsx', f'output/{dt_string}_alertas_x_monto.xlsx']
mail.attach_archives(listado_archive)

: 

In [ ]:
mail.erase_head()

: 

In [ ]:
mail.head_mesagge('usuario@falabella.com.co','usuario@falabella.com.co',f'Reporte diligenciamiento alertas NC - {fecha_inicio} hasta el {fecha_final}','usuario@falabella.com.co')

: 

In [ ]:
mail.send_mesaje()

: 

tiendas

In [ ]:
tienda == 'ACQUA', |
tienda == 'TIENDA ALEGRA'    
tienda == 'ARKADIA', |
tienda == 'BUENAVISTA', |
tienda == 'CACIQUE', | 
tienda == 'CARACOLI', | 
tienda == 'CENTRO MAYOR', | 
tienda == 'COLINA', | 
tienda == 'DIVER PLAZA', | 
tienda == 'EL CASTILLO', | 
tienda == 'FONTANAR', |
tienda == 'GALERIAS', |
tienda == 'HAYUELOS', | 
tienda == 'JARDIN PLAZA', | 
tienda == 'LA CAROLA', | 
tienda == 'LA FELICIDAD', | 
tienda == 'PARQUE ARBOLEDA', | 
tienda == 'PLAZA CENTRAL', | 
tienda == 'PRIMAVERA', | 
tienda == 'SAN DIEGO', | 
tienda == 'SANTA FE', | 
tienda == 'SANTAFE MEDELLIN', | 
tienda == 'SUBA', | 
tienda == 'TITAN', | 
tienda == 'UNICENTRO', |
tienda == 'VILLA COUNTRY', | 
tienda == 'WTC CALI'

: 

In [ ]:
tienda == 'EL CASTILLO' | tienda == 'FONTANAR' |tienda == 'GALERIAS' |tienda == 'HAYUELOS' | tienda == 'JARDIN PLAZA' | tienda == 'LA CAROLA' | tienda == 'LA FELICIDAD' | tienda == 'PARQUE ARBOLEDA' | tienda == 'PLAZA CENTRAL' | tienda == 'PRIMAVERA' | tienda == 'SAN DIEGO' | tienda == 'SANTA FE' | tienda == 'SANTAFE MEDELLIN' | tienda == 'SUBA' | tienda == 'TITAN' | tienda == 'UNICENTRO' |tienda == 'VILLA COUNTRY' | tienda == 'WTC CALI'

: 